<a href="https://colab.research.google.com/github/olexandr7/numismatic_data_tallinn_city_museum/blob/main/numismatic_data_muis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''
=====================================================
LIDO XML → Pandas DataFrame Converter for MUIS Records
=====================================================

This script parses a LIDO XML file from the Estonian museum database (MUIS)
and extracts key metadata fields into a clean pandas DataFrame, which is
then exported to a UTF-8 CSV file.

Each column represents a specific property of the museum object.

──────────────────────────────
COLUMN MAPPING DESCRIPTION
──────────────────────────────

RecID
    Unique local record identifier of the museum object (e.g. 1569464).
    Comes from <lido:lidoRecID lido:type="local">.

Link
    Direct link to the MUIS museum database entry, built as:
    https://www.muis.ee/museaalview/{RecID}

Muuseumikogu
    Museum collection or category (e.g. "numismaatika").
    Extracted from <lido:classification lido:type="muuseumikogu">.

Number
    Full museum inventory number (e.g. "TLM_401 N 1:60").
    From <lido:workID lido:type="museaali täisnumber">.

Nimetus
    Official title or name of the object (e.g. "1 kopikas").
    From <lido:titleSet lido:type="museaali nimetus">.

Olemus
    Type or nature of the object (e.g. "münt", "medal").
    From <lido:objectWorkType lido:type="olemus">.

Dateering
    Production or dating period of the object.
    Uses <earliestDate> and <latestDate> — shows a range ("1990–2000")
    or a single year ("1771") if both are equal.

Originaal
    Raw descriptive note about creation, often describing authenticity
    or replication (from <lido:eventDescriptionSet/lido:descriptiveNoteValue>).

Originaal_tüüp
    Derived field showing whether the item is "originaal" or "koopia".
    Defaults to "originaal" unless the description explicitly mentions "koopia".

Kommentaar
    Additional notes or internal comments, taken from
    <lido:objectDescriptionSet/lido:descriptiveNoteValue> (e.g. "836/76").

Seisund
    Object’s physical condition (e.g. "hea", "rahuldav").
    From <lido:displayStateEditionWrap/lido:displayState>.

Tehnika
    Manufacturing technique(s) used, extracted from
    <lido:termMaterialsTech lido:type="tehnika">,
    ignoring category headers and keeping only actual methods (e.g. "vermimine").

Materjal
    Primary material of the object, taken as the most specific term (last one)
    from <lido:termMaterialsTech lido:type="materjal"> (e.g. "vask", "kollane metall").

Mõõdud
    Object measurements combined into one line (e.g. "läbimõõt 2.4 cm"),
    from <measurementType>, <measurementValue>, and <measurementUnit>.

Riik
    Country of origin or context, from <lido:eventPlace lido:politicalEntity="riik">.
    The first non-empty value is used (e.g. "Venemaa").
'''

In [85]:
import xml.etree.ElementTree as ET
import pandas as pd
#downloading files from Colab
from google.colab import files

# Path to your XML file | just name if stored in Colab
xml_file = "lido2.xml"
ns = {"lido": "http://www.lido-schema.org"}

# Parse the XML
tree = ET.parse(xml_file)
root = tree.getroot()

In [86]:
rows = []
for record in root.findall("lido:lido", ns):
    recid = record.findtext(".//lido:lidoRecID[@lido:type='local']", namespaces=ns)
    muuseumikogu = record.findtext(".//lido:classification[@lido:type='muuseumikogu']/lido:term", namespaces=ns)
    number = record.findtext(".//lido:workID[@lido:type='museaali täisnumber']", namespaces=ns)
    nimetus = record.findtext(".//lido:titleSet[@lido:type='museaali nimetus']/lido:appellationValue", namespaces=ns)
    olemus = record.findtext(".//lido:objectWorkType[@lido:type='olemus']/lido:term", namespaces=ns)
    seisund = record.findtext(".//lido:displayStateEditionWrap/lido:displayState", namespaces=ns)
# --- Dateering ---
    date_start = record.findtext(".//lido:eventDate/lido:date/lido:earliestDate", namespaces=ns)
    date_end = record.findtext(".//lido:eventDate/lido:date/lido:latestDate", namespaces=ns)
    if date_start and date_end:
        dateering = date_start if date_start == date_end else f"{date_start}–{date_end}"
    else:
        dateering = date_start or date_end

# --- Originaal + Originaal_tüüp ---
    originaal = record.findtext(".//lido:eventDescriptionSet/lido:descriptiveNoteValue", namespaces=ns)
    if originaal and "koopia" in originaal.lower():
        originaal_tüüp = "koopia"
    else:
        # Default to 'originaal' if not explicitly a copy or field missing
        originaal_tüüp = "originaal"

# --- Kommentaar (from objectDescriptionSet) ---
    kommentaar = record.findtext(".//lido:objectDescriptionSet/lido:descriptiveNoteValue", namespaces=ns)

# --- Tehnika (skip first two terms) ---
    tehnika_terms = []
    for block in record.findall(".//lido:termMaterialsTech[@lido:type='tehnika']", ns):
        terms = [t.text.strip() for t in block.findall("lido:term", ns) if t.text]
        if len(terms) > 2:
            tehnika_terms.extend(terms[2:])
        elif len(terms) > 1:
            tehnika_terms.extend(terms[1:])
    tehnika = "; ".join(dict.fromkeys(tehnika_terms)) if tehnika_terms else None

# --- Materjal (only most specific term) ---
    materjal_terms = []
    for block in record.findall(".//lido:termMaterialsTech[@lido:type='materjal']", ns):
        terms = [t.text.strip() for t in block.findall("lido:term", ns) if t.text]
        if terms:
            materjal_terms.append(terms[-1])
    materjal = "; ".join(dict.fromkeys(materjal_terms)) if materjal_terms else None

# --- Mõõdud ---
    mõõdud_type = record.findtext(".//lido:measurementType", namespaces=ns)
    mõõdud_val = record.findtext(".//lido:measurementValue", namespaces=ns)
    mõõdud_unit = record.findtext(".//lido:measurementUnit", namespaces=ns)
    mõõdud = f"{mõõdud_type} {mõõdud_val} {mõõdud_unit}" if mõõdud_type and mõõdud_val and mõõdud_unit else mõõdud_val

# --- Riik (first non-empty under politicalEntity="riik") ---
    riik = None
    for el in record.findall(".//lido:eventPlace/lido:place[@lido:politicalEntity='riik']/lido:namePlaceSet/lido:appellationValue", ns):
        if el is not None and el.text:
            cand = el.text.strip()
            if cand and cand != "[]":
                riik = cand
                break

# --- Link ---
    link = f"https://www.muis.ee/museaalview/{recid}" if recid else None

# ---------------------------------------------------------------------------------------------
    rows.append({
        "RecID": recid,
        "Link": link,
        "Muuseumikogu": muuseumikogu,
        "Number": number,
        "Nimetus": nimetus,
        "Olemus": olemus,
        "Dateering": dateering,
        "Originaal": originaal,
        "Originaal_tüüp": originaal_tüüp,
        "Kommentaar": kommentaar,
        "Seisund": seisund,
        "Tehnika": tehnika,
        "Materjal": materjal,
        "Mõõdud": mõõdud,
        "Riik": riik
    })

# Create DataFrame
df = pd.DataFrame(rows)


In [87]:
# Save to CSV
df.to_csv("numismatic_collection_tlm.csv", index=False, encoding="utf-8-sig")
print(f"✅ Done! Saved {len(df)} records to 'numismatic_collection_tlm.csv'")
files.download('numismatic_collection_tlm.csv')

✅ Done! Saved 8700 records to 'numismatic_collection_tlm.csv'


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Printing first row
print(df.iloc[0])
# Printing 40 rows
# print(df.head(40).to_string())

In [ ]:
# Quick data analysis - all column names followed by its unique values
for col in df.columns:
    unique_vals = df[col].dropna().unique()
    print(f"\n--- {col} ---")
    print(unique_vals)